In [1]:
import joblib
import nltk
import pandas as pd
import torch
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report
from tqdm import tqdm

In [2]:
# Load the test CSV
test_csv_path = r"C:\Users\user\OneDrive - National University of Singapore\Desktop\Y4S1\DSA4264\DSA4264-Detoxify\model-1\classical_ml\test_set4.csv"
df = pd.read_csv(test_csv_path)

In [3]:
# Download necessary NLTK data
nltk.download("punkt")
nltk.download("wordnet")

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()


def preprocess(text):
    # Tokenization
    words = word_tokenize(text.lower())
    # Remove punctuation and non-alphabetic tokens
    words = [word for word in words if word.isalpha()]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Set the column names
text_column = "text"  # Column containing the text data
true_labels_column = "Classification"  # Column containing the true labels

# Define multiple model and vectorizer paths
model_paths = [
    r"C:\Users\user\OneDrive - National University of Singapore\Desktop\Y4S1\DSA4264\DSA4264-Detoxify\model-1\classical_ml\ridge_classifier_model.pkl",
    r"C:\Users\user\OneDrive - National University of Singapore\Desktop\Y4S1\DSA4264\DSA4264-Detoxify\model-1\classical_ml\ridge_classifier_model_5k.pkl",
    r"C:\Users\user\OneDrive - National University of Singapore\Desktop\Y4S1\DSA4264\DSA4264-Detoxify\model-1\classical_ml\best_xgb_model.pkl",
]

vectorizer_paths = [
    r"C:\Users\user\OneDrive - National University of Singapore\Desktop\Y4S1\DSA4264\DSA4264-Detoxify\model-1\classical_ml\vectorizer_final.pkl",
    r"C:\Users\user\OneDrive - National University of Singapore\Desktop\Y4S1\DSA4264\DSA4264-Detoxify\model-1\classical_ml\vectorizer_5k.pkl",
    r"C:\Users\user\OneDrive - National University of Singapore\Desktop\Y4S1\DSA4264\DSA4264-Detoxify\model-1\classical_ml\vectorizer_final.pkl",
]

# Class labels (ensure these match the numerical labels in your true_labels_column)
label_mapping = {
    0: "No Hate/Toxic",
    1: "Toxic 1",
    2: "Toxic 2",
    3: "Toxic 3",
    4: "Hate 1",
    5: "Hate 2",
    6: "Hate 3",
}

# Reverse mapping for predictions
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

In [5]:
# Extract text data and preprocess with progress bar
texts = df[text_column].tolist()
processed_samples = [preprocess(text) for text in tqdm(texts, desc="Preprocessing")]

# Predict with each model and vectorizer
for i, (model_path, vectorizer_path) in enumerate(
    zip(model_paths, vectorizer_paths), start=1
):
    print(f"\nEvaluating Model {i}...")

    # Load the model and vectorizer
    clf = joblib.load(model_path)
    vectorizer = joblib.load(vectorizer_path)
    print(f"Model {i} and vectorizer loaded successfully.")

    # Vectorize the input
    new_input_vectorized = vectorizer.transform(processed_samples)

    # Predict using the loaded model with progress bar
    predictions = []
    for vec_sample in tqdm(
        new_input_vectorized, desc=f"Predicting with Model {i}", unit="sample"
    ):
        pred = clf.predict(vec_sample)  # Predict each sample individually
        predictions.append(pred[0])  # Append the prediction result

    # Convert predicted labels to match the format of true labels
    predicted_labels = [
        reverse_label_mapping[label_mapping[pred]] for pred in predictions
    ]

    # Store predictions in DataFrame
    df[f"predicted_labels_model_{i}"] = predicted_labels

    # Generate and print classification report
    print(f"\nClassification Report for Model {i}:\n")
    print(
        classification_report(
            df[true_labels_column],
            df[f"predicted_labels_model_{i}"],
            target_names=list(label_mapping.values()),
        )
    )

Preprocessing: 100%|██████████| 15750/15750 [00:04<00:00, 3833.94it/s]



Evaluating Model 1...
Model 1 and vectorizer loaded successfully.


Predicting with Model 1: 15750sample [00:01, 8817.91sample/s]



Classification Report for Model 1:

               precision    recall  f1-score   support

No Hate/Toxic       0.62      0.74      0.67      2250
      Toxic 1       0.54      0.60      0.57      2250
      Toxic 2       0.90      0.69      0.78      2250
      Toxic 3       1.00      0.98      0.99      2250
       Hate 1       0.73      0.75      0.74      2250
       Hate 2       0.99      0.94      0.97      2250
       Hate 3       1.00      0.98      0.99      2250

     accuracy                           0.81     15750
    macro avg       0.83      0.81      0.81     15750
 weighted avg       0.83      0.81      0.81     15750


Evaluating Model 2...
Model 2 and vectorizer loaded successfully.


Predicting with Model 2: 15750sample [00:01, 9063.97sample/s]



Classification Report for Model 2:

               precision    recall  f1-score   support

No Hate/Toxic       0.53      0.70      0.61      2250
      Toxic 1       0.46      0.42      0.44      2250
      Toxic 2       0.41      0.33      0.36      2250
      Toxic 3       0.95      0.98      0.97      2250
       Hate 1       0.72      0.72      0.72      2250
       Hate 2       0.99      0.94      0.96      2250
       Hate 3       0.99      0.99      0.99      2250

     accuracy                           0.72     15750
    macro avg       0.72      0.72      0.72     15750
 weighted avg       0.72      0.72      0.72     15750


Evaluating Model 3...
Model 3 and vectorizer loaded successfully.


Predicting with Model 3: 15750sample [00:09, 1595.30sample/s]


Classification Report for Model 3:

               precision    recall  f1-score   support

No Hate/Toxic       0.55      0.77      0.65      2250
      Toxic 1       0.53      0.54      0.54      2250
      Toxic 2       0.89      0.67      0.77      2250
      Toxic 3       0.99      0.98      0.98      2250
       Hate 1       0.77      0.69      0.73      2250
       Hate 2       0.99      0.94      0.96      2250
       Hate 3       1.00      0.98      0.99      2250

     accuracy                           0.80     15750
    macro avg       0.82      0.80      0.80     15750
 weighted avg       0.82      0.80      0.80     15750

